In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from ipynb.fs.full.f_dist_jobzone_helper import calc_jobzone_dist_emp

In [4]:
def calc_jobzone_impact(sc,analysis):
    scenario = "StdScen21_"+sc+"_annual_state_"+analysis
    JEDI_name = pd.read_excel(r"../Census/census_to_JEDI.xlsx",sheet_name = "JEDI")
    JEDI_result = pd.read_csv(r"../ReEDS_to_Jobs/output/JEDI_output_"+scenario+".csv") # CHANGE THIS
    
    list_of_states = JEDI_result["state"].unique()
    for m in list_of_states:
        these_jobs = calc_jobzone_dist_emp(JEDI_result,m)
        these_jobs["state"] = m
        if m == list_of_states[0]:
            all_jobs_zones = these_jobs
        else:
            all_jobs_zones = all_jobs_zones.append(these_jobs)
    
    bls_age = pd.read_csv(r"../BLS_Age/BLS_Age_by_Industry.csv")
    
    mismatch_jobs_consolidate = all_jobs_zones.copy().reset_index()
    mismatch_jobs_consolidate["Retire"] = 0
    for this_sector in mismatch_jobs_consolidate["index"].unique():
        this_ret_age = bls_age.query("JEDI_Industries == @this_sector")
        ret_perc = this_ret_age["35 to 44"].iloc[0] + this_ret_age["45 to 54"].iloc[0] + this_ret_age["55 to 64"].iloc[0] + this_ret_age["65 over"].iloc[0]
        mismatch_jobs_consolidate["Retire"][mismatch_jobs_consolidate["index"] == this_sector] = mismatch_jobs_consolidate[2022] * ret_perc
    mismatch_jobs_consolidate["Remaining"] = mismatch_jobs_consolidate[2022] - mismatch_jobs_consolidate["Retire"]
    mismatch_jobs_consolidate["Mismatch_1Y"] = mismatch_jobs_consolidate[2050] - mismatch_jobs_consolidate["Remaining"]
    mismatch_jobs_consolidate["Mismatch_5Y"] = mismatch_jobs_consolidate[[2046,2047,2048,2049,2050]].mean(axis=1) - mismatch_jobs_consolidate["Remaining"]
    mismatch_jobs_consolidate["Mismatch_10Y"] = mismatch_jobs_consolidate[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1) - mismatch_jobs_consolidate["Remaining"]

    detailed_jobzones = mismatch_jobs_consolidate.copy().set_index("index")
    detailed_jobzones = detailed_jobzones[[2022,2050,"Type","state","Retire","Remaining","Mismatch_1Y","Mismatch_5Y","Mismatch_10Y"]]
    detailed_jobzones["MismatchMult_1y"] = detailed_jobzones["Mismatch_1Y"]/detailed_jobzones[2022]
    detailed_jobzones["MismatchMult_1y"] = detailed_jobzones["MismatchMult_1y"].replace(np.nan,0)
    detailed_jobzones["MismatchMult_5y"] = detailed_jobzones["Mismatch_5Y"]/detailed_jobzones[2022]
    detailed_jobzones["MismatchMult_5y"] = detailed_jobzones["MismatchMult_5y"].replace(np.nan,0)
    detailed_jobzones["MismatchMult_10y"] = detailed_jobzones["Mismatch_10Y"]/detailed_jobzones[2022]
    detailed_jobzones["MismatchMult_10y"] = detailed_jobzones["MismatchMult_10y"].replace(np.nan,0)
    detailed_jobzones.to_csv(r"output/Dist_detailedjobzones_"+scenario+".csv")

In [4]:
"""
sc = "Low_RE_Cost_95_by_2035"
analysis = "v6.2jedi_mid"
scenario = "StdScen21_"+sc+"_annual_state_"+analysis
JEDI_name = pd.read_excel(r"../Census/census_to_JEDI.xlsx",sheet_name = "JEDI")
JEDI_result = pd.read_csv(r"../ReEDS_to_Jobs/output/JEDI_output_"+scenario+".csv") # CHANGE THIS
    
list_of_states = JEDI_result["state"].unique()
for m in list_of_states:
    these_jobs = calc_jobzone_dist_emp(JEDI_result,m)
    these_jobs["state"] = m
    if m == list_of_states[0]:
        all_jobs_zones = these_jobs
    else:
        all_jobs_zones = all_jobs_zones.append(these_jobs)
    
bls_age = pd.read_csv(r"../BLS_Age/BLS_Age_by_Industry.csv")

mismatch_jobs_consolidate = all_jobs_zones.copy().reset_index()
mismatch_jobs_consolidate["Retire"] = 0
for this_sector in mismatch_jobs_consolidate["index"].unique():
    this_ret_age = bls_age.query("JEDI_Industries == @this_sector")
    ret_perc = this_ret_age["35 to 44"].iloc[0] + this_ret_age["45 to 54"].iloc[0] + this_ret_age["55 to 64"].iloc[0] + this_ret_age["65 over"].iloc[0]
    mismatch_jobs_consolidate["Retire"][mismatch_jobs_consolidate["index"] == this_sector] = mismatch_jobs_consolidate[2022] * ret_perc
mismatch_jobs_consolidate["Remaining"] = mismatch_jobs_consolidate[2022] - mismatch_jobs_consolidate["Retire"]
mismatch_jobs_consolidate["Mismatch_1Y"] = mismatch_jobs_consolidate[2050] - mismatch_jobs_consolidate["Remaining"]
mismatch_jobs_consolidate["Mismatch_5Y"] = mismatch_jobs_consolidate[[2046,2047,2048,2049,2050]].mean(axis=1) - mismatch_jobs_consolidate["Remaining"]
mismatch_jobs_consolidate["Mismatch_10Y"] = mismatch_jobs_consolidate[[2041,2042,2043,2044,2045,2046,2047,2048,2049,2050]].mean(axis=1) - mismatch_jobs_consolidate["Remaining"]
    
detailed_jobs = mismatch_jobs_consolidate.copy().set_index("index")
detailed_jobs = detailed_jobs[[2022,2050,"Type","state","Retire","Remaining","Mismatch_1Y","Mismatch_5Y","Mismatch_10Y"]]
detailed_jobs["MismatchMult_1y"] = detailed_jobs["Mismatch_1Y"]/detailed_jobs[2022]
detailed_jobs["MismatchMult_1y"] = detailed_jobs["MismatchMult_1y"].replace(np.nan,0)
detailed_jobs["MismatchMult_5y"] = detailed_jobs["Mismatch_5Y"]/detailed_jobs[2022]
detailed_jobs["MismatchMult_5y"] = detailed_jobs["MismatchMult_5y"].replace(np.nan,0)
detailed_jobs["MismatchMult_10y"] = detailed_jobs["Mismatch_10Y"]/detailed_jobs[2022]
detailed_jobs["MismatchMult_10y"] = detailed_jobs["MismatchMult_10y"].replace(np.nan,0)
#detailed_jobs.to_csv(r"output/Dist_detailedjobzones_"+scenario+".csv")
"""